In [1]:
from openai import OpenAI
import pandas as pd

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
client = OpenAI()

In [4]:
url = "https://raw.githubusercontent.com/Choonholic/jpub_ai_agent/main/codes/chapter%2004/4.4%20Presentations/data/sales_data.csv"
sales_data = pd.read_csv(url)

In [6]:
sales_data.head()

,날짜,기초부터 배우는 기계 학습,태진의 10가지 데이터 분석,GPT 그림 해설
0,31/3/2022,54545,177410,163757
1,30/6/2022,90880,156803,184146
2,30/9/2022,103810,186471,194499
3,31/12/2022,120690,192272,206815
4,31/3/2023,138022,129303,216851


In [7]:
import requests
import io

resp = requests.get(url)
resp.raise_for_status()
bio = io.BytesIO(resp.content)

file = client.files.create(
    file=bio,
    purpose='assistants'
)

In [10]:
assistant = client.beta.assistants.create(
    instructions="데이터 과학 도우미로서, 주어진 데이터와 요청에 따라 적절한 코드를 작성하고 적절한 시각화를 생성할 수 있습니다.",
    model="gpt-4o-mini",
    tools=[
        {'type': 'code_interpreter'}
    ],
    tool_resources={
        'code_interpreter': {
            'file_ids': [file.id]
        }
    }
)

print(assistant)

Assistant(id='asst_YeXa5vitKyk3UHftiY37phiG', created_at=1763206147, description=None, instructions='데이터 과학 도우미로서, 주어진 데이터와 요청에 따라 적절한 코드를 작성하고 적절한 시각화를 생성할 수 있습니다.', metadata={}, model='gpt-4o-mini', name=None, object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=['file-AVTxcuNC3QMeSyv3PfpV77']), file_search=None), top_p=1.0, reasoning_effort=None)


In [16]:
thread = client.beta.threads.create(
    messages=[
        {
            'role': 'user',
            'content': '2022년부터 2025년까지 각 분기의 총 판매액을 계산하고, 이를 다른 제품으로 시각화하여 선 그래프로 표시하세요. 제품의 선 색상은 각각 빨강, 파랑, 녹색으로 설정하세요.',
            'attachments': [
                {
                    'file_id': file.id,
                    'tools': [
                        {'type': 'code_interpreter'}
                    ]
                }
            ]
        }
    ]
)

print(thread)

/var/folders/9b/zsx19sgj6png2ps_b4wlyx680000gn/T/ipykernel_9301/2647666130.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create(


Thread(id='thread_dXZ54GpbeUAyv0NKtzJoOnZU', created_at=1763206631, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=['file-AVTxcuNC3QMeSyv3PfpV77']), file_search=None))


In [17]:
run = client.beta.threads.with_raw_response.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

print(run)

<APIResponse [200 OK] type=<class 'openai.types.beta.threads.run.Run'>>
